In [1]:
try:
  import re
  import json
  from IPython.core.display import HTML
  import pandas as pd
  from base64 import b64decode
  from bs4 import BeautifulSoup as bs
  import markdown2
except ModuleNotFoundError as e:
  %pip install pandas
  %pip install beautifulsoup4
  %pip install markdown2

In [20]:
fn = 'jacbian.ipynb'
with open(fn) as f:
  js = json.load(f)['cells']
  df = pd.DataFrame(js)

# Caveat
### For all this to work text and math markdown should be seperated into seperate cells

In [21]:
# templete for markdown only notebooks
mrkdn_tmpl = '{"cells": [{"cell_type": "markdown", "metadata": {}, "source": []}], "metadata": {"language_info": {"name": "python"}, "orig_nbformat": 4}, "nbformat": 4, "nbformat_minor": 2}'
texCapsul_s = '\n<div><script type="math/tex; mode=display" id="MathJax-Element-1">'
texCapsul_e = '</script></div>\n'
codeCapsule_s = "\n<div><pre><code class='language-python'>"
codeCapsule_e = "</code></pre></div>\n"
regex = "(<?p?>?(\\\\begin{equation[*]?}.*?{equation[*]?})\<\/p\>)"

In [22]:
def writePNG(txt, filename):
  try:
    with open(filename, 'wb') as f:
      f.write(b64decode(txt))
  except Exception as e:
    return False
  return True

In [74]:
def extract_d(item):
    try:
        dcts = []
        for el in item:
            dcts.append([el['data'],el['output_type']])
        return dcts
    except Exception as e:
        return  None

def encapsMath(match):
    if len(match.groups()) > 0:
        return f"\n{texCapsul_s + match.group().strip()}\n"
    else:
        return f"\n{match.group().strip()}\n"

def convert(row):
    if row['cell_type'] == 'code':
        html = ""
        html += codeCapsule_s
        html += row['sourceCLN']
        html += codeCapsule_e
        return bs(html)
        # if row['outputsCLN'] !=False:
        #     #TODO
        #     pass
    elif row['cell_type'] == 'markdown':
        txt = row['sourceCLN']
        txt = markdown2.markdown(txt).replace(
                '</em>', '').replace(
                '<em>', '')
        p = re.compile(regex)
        txt = p.sub(encapsMath, txt)
        
        return bs(txt)
def clean(lst):
  lst = list(filter(lambda x: x!='',[x.strip() for x in lst]))
  return lst


In [79]:
df['sourceCLN'] = df['source'].apply(clean).str.join('\n')
df['outputsCLN'] = df['outputs'].fillna(False).apply(
  lambda x: x if type(x)!=bool and len(x)>0 else False)
df['html'] = df.apply(convert,axis=1)
df.head()

,attachments,cell_type,metadata,source,execution_count,outputs,sourceCLN,outputsCLN,html
0,{},markdown,{},[### Understanding Jacobian functions through ...,NaN,NaN,### Understanding Jacobian functions through e...,False,[[Understanding Jacobian functions through exp...
1,NaN,code,{},"[try: \n, from sympy import *\n, except Mod...",10.0,"[{'name': 'stdout', 'output_type': 'stream', '...",try:\nfrom sympy import *\nexcept ModuleNotFou...,"[{'name': 'stdout', 'output_type': 'stream', '...","[\n, [[<code class=""language-python"">try:\nfro..."
2,{},markdown,{},[### Working Out the Jacobian for an example f...,NaN,NaN,### Working Out the Jacobian for an example fu...,False,[[Working Out the Jacobian for an example func...
3,{},markdown,{},"[\begin{equation*}\n, f(x) = x^{2}y+3z\n, \end...",NaN,NaN,\begin{equation*}\nf(x) = x^{2}y+3z\n\end{equa...,False,[[\begin{equation}\nf(x) = x^{2}y+3z\n\end{equ...
4,{},markdown,{},[The partial derivatives of the function with ...,NaN,NaN,The partial derivatives of the function with r...,False,[[The partial derivatives of the function with...


In [83]:
html = df.html.iloc[34]

<h4>Solution for Q2</h4>
<p>\begin{equation}
u(r,\theta,\phi) = r \sin{\left(\phi \right)} \cos{\left(\theta \right)}
\end{equation}
\begin{equation}
v(r,\theta,\phi) = r \sin{\left(\phi \right)} \sin{\left(\theta \right)}
\end{equation}
\begin{equation}
w(r,\theta,\phi) = r \cos{\left(\phi \right)}
\end{equation}
hence the derived Jacobian is</p>
<hr/>
<p>\begin{equation}
J =
\begin{bmatrix}
\sin{\left(\phi \right)} \cos{\left(\theta \right)} &amp;
- r \sin{\left(\phi \right)} \sin{\left(\theta \right)} &amp;
r \cos{\left(\phi \right)} \cos{\left(\theta \right)} \
\
\sin{\left(\phi \right)} \sin{\left(\theta \right)} &amp;
r \sin{\left(\phi \right)} \cos{\left(\theta \right)} &amp;
r \sin{\left(\theta \right)} \cos{\left(\phi \right)} \
\
\cos{\left(\phi \right)} &amp;
0 &amp;
- r \sin{\left(\phi \right)}
\end{bmatrix}
\end{equation}
Evaluating the Jacobian at $r = 0$, $\theta = 0$, $\phi = 0$ gives
\begin{equation}
\begin{bmatrix}
0 &amp; 0 &amp; 0 \
0 &amp; 0 &amp; 0\
0 &amp; 0 &amp

In [62]:
with open('jacobian.html','w') as f:
    f.writelines('\n'.join(df['html'].values))

In [49]:
df[['html','cell_type']].head()

,html,cell_type
0,<h3>Understanding Jacobian functions through e...,markdown
1,\n<div><pre><code class='language-python'>try:...,code
2,<h3>Working Out the Jacobian for an example fu...,markdown
3,<p>\begin{equation}\nf(x) = x^{2}y+3z\n\end{eq...,markdown
4,<p>The partial derivatives of the function wit...,markdown


In [50]:
p = re.compile(regex)
print(p.sub(encapsMath, x))

\begin{equation*}
\frac{\partial f}{\partial x} = e^{x} \cos{\left(y \right)} + e^{3 y}
\end{equation*}
\begin{equation*}
\frac{\partial f}{\partial y} = x^{2} + \frac{3}{4}x
\end{equation*}


In [68]:
html = bs('\n'.join(df.html))

In [73]:
html.contents[-2]

<p>\begin{equation}
J =
\begin{bmatrix}
\sin{\left(\phi \right)} \cos{\left(\theta \right)} &amp;
- r \sin{\left(\phi \right)} \sin{\left(\theta \right)} &amp;
r \cos{\left(\phi \right)} \cos{\left(\theta \right)} \
\
\sin{\left(\phi \right)} \sin{\left(\theta \right)} &amp;
r \sin{\left(\phi \right)} \cos{\left(\theta \right)} &amp;
r \sin{\left(\theta \right)} \cos{\left(\phi \right)} \
\
\cos{\left(\phi \right)} &amp;
0 &amp;
- r \sin{\left(\phi \right)}
\end{bmatrix}
\end{equation}
Evaluating the Jacobian at $r = 0$, $\theta = 0$, $\phi = 0$ gives
\begin{equation}
\begin{bmatrix}
0 &amp; 0 &amp; 0 \
0 &amp; 0 &amp; 0\
0 &amp; 0 &amp; 1
\end{bmatrix}
\end{equation}
Evaluating the Jacobian at $r = 1$, $\theta = 0$, $\phi = 0$ gives
\begin{equation}
\begin{bmatrix}
0 &amp; 0 &amp; 1 \
0 &amp; 0 &amp; 2\
0 &amp; 0 &amp; 1
\end{bmatrix}
\end{equation}
Evaluating the Jacobian at $r = 1$, $\theta = \frac{\pi}{4}$, $\phi = \frac{\pi}{4}$ gives
\begin{equation}
\begin{bmatrix}
\frac{1}{2} &